In [14]:
import warnings
import sklearn.metrics as metrics

In [2]:
warnings.filterwarnings("ignore")

In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

Using TensorFlow backend.


In [4]:
from dataset import suny_international

In [5]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [6]:
def vanilla_lstm(n_steps, n_features):
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [7]:
n_steps=20
n_features= 1

In [8]:
df=suny_international.load_data()['GHI']
df_train,df_test = suny_international.train_test_split(df)

In [9]:
X, y = split_sequence(df_train, n_steps)
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [12]:
model=vanilla_lstm(n_steps, n_features)
history=model.fit(X,y,epochs=40,verbose=True)
model.save('models/simple_rnn.h5')
print(history.history['loss'][-1])

Epoch 1/40
22996/22996 [==============================] - 22s 968us/step - loss: 46319.8493
Epoch 2/40
22996/22996 [==============================] - 23s 1ms/step - loss: 45683.2562
Epoch 3/40
22996/22996 [==============================] - 21s 935us/step - loss: 14478.5046
Epoch 4/40
22996/22996 [==============================] - 21s 927us/step - loss: 16007.2480
Epoch 5/40
22996/22996 [==============================] - 22s 965us/step - loss: 14348.9271
Epoch 6/40
22996/22996 [==============================] - 22s 963us/step - loss: 12587.0579
Epoch 7/40
22996/22996 [==============================] - 21s 920us/step - loss: 12114.2940
Epoch 8/40
22996/22996 [==============================] - 21s 922us/step - loss: 11459.9827
Epoch 9/40
22996/22996 [==============================] - 23s 995us/step - loss: 11432.6040
Epoch 10/40
22996/22996 [==============================] - 18s 787us/step - loss: 10938.9952
Epoch 11/40
22996/22996 [==============================] - 21s 926us/step - loss:

In [15]:
X, y = split_sequence(df_test, n_steps)
X = X.reshape((X.shape[0], X.shape[1], n_features))
y_pred = model.predict(X)
metrics.mean_squared_error(y_true=y,y_pred=y_pred)

11814.849850604623